In [36]:
#on travaille sur le json issu du scraping de jp
import pandas as pd
import json

#chemin du json
json_file_path = 'melo/jpboxoffice/moviesdata.json'

# charger json
with open(json_file_path) as f:
    data = json.load(f)

# json devient df
films = pd.json_normalize(data)

#  df convertit en CSV
films.to_csv('moviedata_test.csv', index=False)
films.dtypes

url            object
titre          object
realisateur    object
pays           object
date           object
genre          object
studio         object
casting        object
franchise      object
remake         object
budget         object
acteurs        object
producteur     object
compositeur    object
dtype: object

In [37]:
#normalisation à la main (même principe que minmaxscaler)
actors = pd.read_csv('acteurs.csv')

actors['prix_normal'] = (actors['prix'] - actors['prix'].min()) / (actors['prix'].max() - actors['prix'].min())

print(actors)
actors['prix_normal'].max()

     duree_carriere                  name  nombre_tournage  prix  prix_normal
0                40     Leonardo DiCaprio               72     8     0.241379
1                39           Johnny Depp               97     9     0.275862
2                43            Tom Cruise               66    18     0.586207
3                26         Jason Statham               59     2     0.034483
4                16         Margot Robbie               47    15     0.482759
..              ...                   ...              ...   ...          ...
268              50     Denzel Washington               70     8     0.241379
269              28         Jeremy Renner               55     7     0.206897
270              30       Jean-Paul Rouve               79     4     0.103448
271              21  Matthias Schoenaerts               42     5     0.137931
272              59     Samuel L. Jackson              168     4     0.103448

[273 rows x 5 columns]


1.0

In [38]:
df = pd.read_csv('moviesdatatest2.csv')

df.head(2)

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre,url
0,NaN,315 000 000 $,NaN,16/12/2009,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,\r\n James ...,NaN,NaN,\r\n 20th Century Fox,\r\n Avatar,https://www.jpbox-office.com/fichfilm.php?id=9...
1,\r\n Sam Worthingto...,315 000 000 $,James Horner,16/12/2009,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,\r\n James ...,NaN,NaN,\r\n 20th Century Fox,\r\n Avatar,https://www.jpbox-office.com/fichfilm.php?id=9...


In [39]:
# Dropping 'url' and 'salles_premiere_semaine' columns
movies_data_cleaned = df.drop(['url', 'salles_premiere_semaine'], axis=1)

# Keeping rows where both 'acteurs' and 'budget' are not null to avoid incomplete data
movies_data_cleaned = movies_data_cleaned.dropna(subset=['acteurs', 'budget'])

#Dropping duplicate rows based on all columns to ensure unique movie entries
movies_data_cleaned = movies_data_cleaned.drop_duplicates()

movies_data_cleaned.head(2)

movies_data_cleaned.shape

(4370, 13)

In [40]:
###NETTOYAGE REALISATEUR

for index, value in movies_data_cleaned['realisateur'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.replace('-','')
        value = value.strip()
        movies_data_cleaned.at[index, 'realisateur'] = value

movies_data_cleaned.head(5)

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,studio,titre
1,\r\n Sam Worthingto...,315 000 000 $,James Horner,16/12/2009,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,\r\n 20th Century Fox,\r\n Avatar
4,\r\n Daniel Radclif...,130 000 000 $,John Williams,02/06/2004,2 503 699,Franchise,Fantasy,Etats-Unis,NaN,Alfonso Cuaron,NaN,\r\n Warner Bros.,\r\n Harry Potter e...
5,"\r\n Adam Driver,\r...",200 000 000 $,John Williams,13/12/2017,2 510 462,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,Rian Johnson,NaN,\r\n Walt Disney Pictures,\r\n Star Wars: Les...
7,"\r\n Chris Evans,\r...",295 000 000 $,Alan Silvestri,25/04/2018,2 565 953,Franchise,Comicbook,Etats-Unis,Kevin Feige,Russo (brothers),NaN,\r\n Walt Disney Pictures,\r\n Avengers: Infi...
11,\r\n Sam Worthingto...,350 000 000 $,NaN,14/12/2022,2 739 848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,\r\n Walt Disney Pictures,\r\n Avatar : la vo...


In [41]:
###NETTOYAGE TITRE
for index, value in movies_data_cleaned['titre'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.strip()
        movies_data_cleaned.at[index, 'titre'] = value

###NETTOYAGE STUDIO
for index, value in movies_data_cleaned['studio'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.strip()
        movies_data_cleaned.at[index, 'studio'] = value

###NETTOYAGE ACTEURS
movies_data_cleaned['acteurs'] = movies_data_cleaned['acteurs'].str.strip().replace(r"\r\n", "", regex=True).replace(r"\s+", " ", regex=True)

In [42]:
### NETTOYAGE DATE
movies_data_cleaned['date'] = movies_data_cleaned['date'].str.strip()
movies_data_cleaned['date'] = pd.to_datetime(movies_data_cleaned['date'], format='%d/%m/%Y')

movies_data_cleaned.head(2)

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,studio,titre
1,"Sam Worthington, Sigourney Weaver",315 000 000 $,James Horner,2009-12-16,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,20th Century Fox,Avatar
4,"Daniel Radcliffe, Emma Watson, Gary Oldman, Al...",130 000 000 $,John Williams,2004-06-02,2 503 699,Franchise,Fantasy,Etats-Unis,NaN,Alfonso Cuaron,NaN,Warner Bros.,Harry Potter et le prisonnier d'Azkaban


In [43]:
### NETTOYAGE BUDGET
movies_data_cleaned['budget'] = movies_data_cleaned['budget'].str.replace('[\$,€]', '', regex=True).str.replace(' ', '').str.replace('?', '')

# Converting to numeric, coercing errors to NaN, then filling NaNs with 0
movies_data_cleaned['budget'] = pd.to_numeric(movies_data_cleaned['budget'], errors='coerce').fillna(0).astype(int)

movies_data_cleaned.head(2)

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,studio,titre
1,"Sam Worthington, Sigourney Weaver",315000000,James Horner,2009-12-16,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,20th Century Fox,Avatar
4,"Daniel Radcliffe, Emma Watson, Gary Oldman, Al...",130000000,John Williams,2004-06-02,2 503 699,Franchise,Fantasy,Etats-Unis,NaN,Alfonso Cuaron,NaN,Warner Bros.,Harry Potter et le prisonnier d'Azkaban


In [44]:
movies_data_cleaned[movies_data_cleaned.acteurs.str.contains('Sam Worthington')]

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,studio,titre
1,"Sam Worthington, Sigourney Weaver",315000000,James Horner,2009-12-16,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,20th Century Fox,Avatar
11,"Sam Worthington, Sigourney Weaver, Kate Winslet",350000000,NaN,2022-12-14,2 739 848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,Walt Disney Pictures,Avatar : la voie de l'eau
1107,"Arnold Schwarzenegger, Sam Worthington",35000000,NaN,2014-05-07,88 534,NaN,Aventure - Action,Etats-Unis,NaN,NaN,NaN,Metropolitan,Sabotage
1131,"Bruce Willis, Colin Farrell, Sam Worthington",70000000,NaN,2002-05-29,84 139,NaN,Guerre,Etats-Unis,NaN,NaN,NaN,UFD,Mission évasion
2398,Sam Worthington,20000000,NaN,2008-08-13,8 527,NaN,Horreur,Australie,NaN,NaN,NaN,TFM Distribution,Solitaire
2840,"Jennifer Aniston, Sam Worthington, William H. ...",0,NaN,2015-04-08,24 050,NaN,Drame,Etats-Unis,NaN,NaN,NaN,Warner Bros.,Cake
3639,"Sam Worthington, Sigourney Weaver",25000000,James Horner,2010-09-01,53 756,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,20th Century Fox,Avatar (Special Edition)
4544,"Jessica Chastain, Helen Mirren, Sam Worthington",20000000,Thomas Newman,2011-06-15,88 044,NaN,Thriller,Etats-Unis,NaN,NaN,NaN,UIP,L'Affaire Rachel Singer
5893,"Sam Worthington, Ed Harris",42000000,NaN,2012-02-15,156 939,NaN,Thriller,Etats-Unis,Lorenzo di Bonaventura,NaN,NaN,Studiocanal,Dos au mur
6454,"Keira Knightley, Guillaume Canet, Eva Mendes, ...",10000000,NaN,2011-02-16,187 573,NaN,Drame,Etats-Unis,NaN,NaN,NaN,Gaumont,Last Night


In [45]:
###repérer les lignes avec sam worthington 
acteur_sam = movies_data_cleaned.mask(movies_data_cleaned["acteurs"] == "Sam Worthington")
acteur_sam

,acteurs,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,studio,titre
1,"Sam Worthington, Sigourney Weaver",315000000.0,James Horner,2009-12-16,2 648 596,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,20th Century Fox,Avatar
4,"Daniel Radcliffe, Emma Watson, Gary Oldman, Al...",130000000.0,John Williams,2004-06-02,2 503 699,Franchise,Fantasy,Etats-Unis,NaN,Alfonso Cuaron,NaN,Warner Bros.,Harry Potter et le prisonnier d'Azkaban
5,"Adam Driver, Benicio Del Toro, Joseph Gordon-L...",200000000.0,John Williams,2017-12-13,2 510 462,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,Rian Johnson,NaN,Walt Disney Pictures,Star Wars: Les derniers Jedi
7,"Chris Evans, Chris Hemsworth, Josh Brolin, Rob...",295000000.0,Alan Silvestri,2018-04-25,2 565 953,Franchise,Comicbook,Etats-Unis,Kevin Feige,Russo (brothers),NaN,Walt Disney Pictures,Avengers: Infinity War
11,"Sam Worthington, Sigourney Weaver, Kate Winslet",350000000.0,NaN,2022-12-14,2 739 848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,Walt Disney Pictures,Avatar : la voie de l'eau
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9608,"Leonardo DiCaprio, Tom Hanks, Nathalie Baye, A...",52000000.0,John Williams,2003-02-12,1 137 654,NaN,Aventure - Action,Etats-Unis,Steven Spielberg,Steven Spielberg,NaN,UIP,Arrête-moi si tu peux
9609,"Edouard Baer, Gérard Depardieu, Dany Boon, Cat...",61239780.0,NaN,2012-10-17,1 132 279,Franchise,Comédie,France,NaN,Laurent Tirard,NaN,Wild Bunch,Astérix et Obélix: Au service de sa Majesté
9610,"Sandrine Kiberlain, Valérie Lemercier, Kad Mer...",22700000.0,NaN,2009-09-30,1 132 450,NaN,Comédie,France,NaN,Laurent Tirard,NaN,Wild Bunch,Le Petit Nicolas
9611,"Leonardo DiCaprio, Brad Pitt, Dakota Fanning, ...",90000000.0,NaN,2019-08-14,1 132 221,NaN,Thriller,Etats-Unis,NaN,Quentin Tarantino,NaN,Sony Pictures,Once Upon a Time in Hollywood


In [46]:
### SAUVEGARDER DATAFRAME EN CSV
movies_data_cleaned.to_csv('movies_clean_v1.csv', index=False)